<a href="https://colab.research.google.com/github/pratikiiitb2013/EVA/blob/master/Assignment9/Neural_word_embeddings_and_sentiment_analysis_ex1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torchtext import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy')
LABEL = data.LabelField(dtype = torch.float)

In [2]:
from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:07<00:00, 11.2MB/s]


In [3]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of testing examples: 25000


In [4]:
train_data

In [5]:
print(vars(train_data.examples[0]))

{'text': ['I', 'heard', 'tell', 'that', 'Madonna', 'was', 'briefly', 'considered', 'for', 'the', 'Catherine', 'Tremell', 'role', '.', 'Compared', 'to', 'Sharon', 'Stone', ',', 'Madonna', 'is', 'too', 'coarse', 'and', 'BAUERISCH', '.', 'She', "'s", 'not', 'even', 'close', '.', '<', 'br', '/><br', '/>EVIL', 'INCARNATE', ':', 'Sharon', 'Stone', 'is', 'a', 'bit', 'long', 'in', 'the', 'tooth', ',', 'the', 'ameliorative', 'effects', 'of', 'modern', 'chemistry', 'and', 'surgery', 'notwithstanding', '.', 'However', ',', 'she', 'artfully', 'treats', 'us', 'to', 'a', 'frightening', 'personification', 'of', 'evil', 'beyond', 'redemption', '.', 'In', 'the', 'obligatory', 'sex', 'scene', ',', 'she', 'projects', 'pure', ',', 'crystalline', 'lust', '.', 'Especially', 'her', 'hooded', ',', 'luminous', 'eyes', 'and', 'a', 'face', 'flat', 'with', 'pleasure', '.', 'Thanks', 'to', 'brilliant', 'use', 'of', 'lighting', 'and', 'other', 'stage', 'techniques', ',', 'the', 'harsh', 'lines', 'of', 'age', 'are',

In [6]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [7]:

print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [8]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [9]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [10]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 203885), (',', 193527), ('.', 166467), ('a', 110012), ('and', 109731), ('of', 101301), ('to', 93715), ('is', 76698), ('in', 61693), ('I', 53957), ('it', 53378), ('that', 49442), ('"', 44405), ("'s", 43712), ('this', 42411), ('-', 37208), ('/><br', 35622), ('was', 34996), ('as', 30626), ('with', 30172)]


In [11]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'a', 'and', 'of', 'to', 'is']


In [12]:

print(LABEL.vocab.stoi)

print(TEXT.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7fd7d4ccaae8>, {'neg': 0, 'pos': 1})
defaultdict(<function _default_unk_index at 0x7fd7d4ccaae8>, {'<unk>': 0, '<pad>': 1, 'the': 2, ',': 3, '.': 4, 'a': 5, 'and': 6, 'of': 7, 'to': 8, 'is': 9, 'in': 10, 'I': 11, 'it': 12, 'that': 13, '"': 14, "'s": 15, 'this': 16, '-': 17, '/><br': 18, 'was': 19, 'as': 20, 'with': 21, 'movie': 22, 'for': 23, 'film': 24, 'The': 25, 'but': 26, '(': 27, "n't": 28, ')': 29, 'on': 30, 'you': 31, 'are': 32, 'not': 33, 'have': 34, 'his': 35, 'be': 36, 'he': 37, 'one': 38, 'by': 39, '!': 40, 'at': 41, 'all': 42, 'an': 43, 'who': 44, 'they': 45, 'from': 46, 'like': 47, 'so': 48, 'her': 49, 'has': 50, "'": 51, 'or': 52, 'about': 53, 'out': 54, 'just': 55, 'It': 56, 'do': 57, '?': 58, 'good': 59, 'some': 60, 'more': 61, 'very': 62, 'would': 63, 'up': 64, 'what': 65, 'This': 66, 'there': 67, 'time': 68, 'can': 69, 'when': 70, 'which': 71, 'she': 72, 'if': 73, 'had': 74, 'only': 75, 'story': 76, 'really': 77, 'their': 

In [13]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

In [14]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):
        #text = [sent len, batch size]
        embedded = self.embedding(text)
        #embedded = [sent len, batch size, emb dim]
        output, hidden = self.rnn(embedded)
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        return self.fc(hidden.squeeze(0))

In [15]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [16]:
! pip install torchsummaryX
from torchsummaryX import summary
inputs = torch.zeros((512,1), dtype = torch.long) #[ sentence length , batch size]
summary(model, inputs)

             Kernel Shape   Output Shape   Params  Mult-Adds
Layer                                                       
0_embedding  [100, 25002]  [512, 1, 100]  2500200    2500200
1_rnn                   -  [512, 1, 256]    91648      91136
2_fc             [256, 1]         [1, 1]      257        256
------------------------------------------------------------
                       Totals
Total params          2592105
Trainable params      2592105
Non-trainable params        0
Mult-Adds             2591592


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_embedding,"[100, 25002]","[512, 1, 100]",2500200,2500200
1_rnn,-,"[512, 1, 256]",91648,91136
2_fc,"[256, 1]","[1, 1]",257,256


In [17]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,592,105 trainable parameters


In [18]:

import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [19]:
criterion = nn.BCEWithLogitsLoss()


In [20]:
model = model.to(device)
criterion = criterion.to(device)

In [21]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [22]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        # print(batch.text)
        optimizer.zero_grad()
                
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [23]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


In [24]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [25]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 45s
	Train Loss: 0.694 | Train Acc: 50.10%
	 Val. Loss: 0.697 |  Val. Acc: 50.38%
Epoch: 02 | Epoch Time: 0m 45s
	Train Loss: 0.693 | Train Acc: 49.66%
	 Val. Loss: 0.697 |  Val. Acc: 50.43%
Epoch: 03 | Epoch Time: 0m 45s
	Train Loss: 0.693 | Train Acc: 49.91%
	 Val. Loss: 0.697 |  Val. Acc: 51.17%
Epoch: 04 | Epoch Time: 0m 45s
	Train Loss: 0.693 | Train Acc: 49.87%
	 Val. Loss: 0.697 |  Val. Acc: 49.66%
Epoch: 05 | Epoch Time: 0m 45s
	Train Loss: 0.693 | Train Acc: 50.08%
	 Val. Loss: 0.697 |  Val. Acc: 51.20%


In [26]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.709 | Test Acc: 47.71%
